In [2]:
import pandas as pd

TEMPLATE_NAMES = [
    'storytelling',
    'example_perturbation',
    'distracting_constraints',
    'negation_objective',
    'gameified',
    'domain_shift',
]

model_name = 'deepseek_7b'
perturbation_type = 'storytelling'

df = pd.read_csv(f'evaluation_log_{model_name}_{perturbation_type}.csv', index_col=False)
df

,perturbation_type,orig_question_content,modified_question_content,difficulty,model,public_test_cases,output,solution_code,result,metadata_output
0,storytelling,You are given a 0-indexed array of strings det...,**The Mysterious Case of the Aging Passengers*...,easy,deepseek_7b,"[{""input"": ""[\""7868190130M7522\"", \""5303914400...","Okay, I need to solve this problem where I hav...","class Solution:\n def countSeniors(self, de...","[True, True]",{'execution time': 3.5762786865234375e-06}
1,storytelling,You are given a 0-indexed 2D integer array num...,**The Quest for the Golden Number**\n\nIn the ...,medium,deepseek_7b,"[{""input"": ""[[7, 2, 1], [6, 4, 2], [6, 5, 3], ...","Okay, I need to solve this problem where I hav...",from typing import List\n\nclass Solution:\n ...,"[True, True]",{'execution time': 1.0251998901367188e-05}


In [4]:
import ast

def compute_true_percentage(df, result_column="result"):
    total_true = 0
    total_cases = 0

    for row in df[result_column]:
        if isinstance(row, str) and row.startswith("["):
            try:
                result_list = ast.literal_eval(row)
                total_true += sum(result_list)
                total_cases += len(result_list)
            except Exception as e:
                print(f"⚠️ Error parsing row: {row} — {e}")

    accuracy = (total_true / total_cases) if total_cases > 0 else 0
    return total_true, total_cases, accuracy

In [5]:
total_true, total_cases, accuracy = compute_true_percentage(df, result_column="result")

print(f"✅ Total test cases: {total_cases}")
print(f"✅ Total passed (True): {total_true}")
print(f"✅ Overall accuracy: {accuracy:.2%}")

✅ Total test cases: 8
✅ Total passed (True): 8
✅ Overall accuracy: 100.00%


In [35]:
import ast
import pandas as pd

def compute_accuracy_by_difficulty(df):
    results = []

    for difficulty, group in df.groupby("difficulty"):
        total_cases = 0
        total_correct = 0

        for result in group["result"]:
            if isinstance(result, str) and result.startswith("["):
                parsed = ast.literal_eval(result)
                total_cases += len(parsed)
                total_correct += sum(parsed)

        accuracy = total_correct / total_cases if total_cases > 0 else 0
        results.append({
            "difficulty": difficulty,
            "total_cases": total_cases,
            "total_correct": total_correct,
            "accuracy": accuracy
        })

    return pd.DataFrame(results)

# Example usage
difficulty_summary = compute_accuracy_by_difficulty(df)
print(difficulty_summary)


  difficulty  total_cases  total_correct  accuracy
0       easy          173            154  0.890173
1       hard           61             17  0.278689
2     medium          199            128  0.643216
